In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf

import cartopy
from cartopy import crs as ccrs

from bokeh.tile_providers import STAMEN_TONER, STAMEN_TONER_LABELS

hv.extension('bokeh')

In most GeoViews examples, we have selected the matplotlib plotting backend, because it has general support for projecting data to different geographic projections using cartopy. The Bokeh backend offers much more advanced tools to interactively explore data, but is currently restricted to displaying data in web Mercator coordinates. Luckily, cartopy makes it possible to project points, geometries and even images from arbitrary coordinate systems into web Mercator so that they can be rendered by Bokeh. So as long as you choose the web Mercator format for your output (or don't specify the output format), you should be able to use Bokeh for any of the GeoViews examples from other notebooks. Bokeh also provides a general interface to render web-based map tile sources, making it simple to overlay your plots onto map tiles.

# WMTS Tile Sources

When using the matplotlib backend, the ``gv.WMTS`` element accepts tile source URLs valid for cartopy. When using the Bokeh backend, you will need to first wrap the URL into a ``WMTSTileSource`` object, because Bokeh's tile support uses a different URL format. Here we provide a list of common tile sources for use with Bokeh. Additional open tile sources you could use can be found at [openstreetmap.org](http://wiki.openstreetmap.org/wiki/Tile_servers).

In [ ]:
tiles = {'OpenMap': 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png',
         'ESRI': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg',
         'Wikipedia': 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png',
         'Stamen Toner': STAMEN_TONER}

Now we can lay these Elements out in an NdLayout by wrapping each ``WMTSTileSource`` in a ``WMTS`` Element and specifying both the extent and the coordinate reference system of these extents. Note that the extents are only required when displaying the tile source on its own; when it is overlaid with some data the data determines the extent automatically.

In [ ]:
%%opts WMTS [width=450 height=250 xaxis=None yaxis=None]
hv.NdLayout({name: gv.WMTS(wmts, extents=(0, -90, 360, 90), crs=ccrs.PlateCarree())
            for name, wmts in tiles.items()}, kdims=['Source']).cols(2)

A tile source may also be drawn at a different ``level`` allowing us to overlay a regular tile source with a set of labels. Valid options for the 'level' option include 'image', 'underlay', 'glyph', 'annotation' and 'overlay':

In [ ]:
%%opts WMTS [width=600 height=570]
gv.WMTS(tiles['ESRI'], extents=(0, -90, 360, 90), crs=ccrs.PlateCarree()) *\
gv.WMTS(STAMEN_TONER_LABELS).opts(style=dict(level='annotation'))

You may also supply a tuple of tile sources pairing a Bokeh WMTSTileSource object with a simple cartopy string URL, allowing the `gv.WMTS` element to be used for both matplotlib and Bokeh rendering without having to declare separate objects.

## Plotting data

One of the main benefits of plotting data with Bokeh is the interactivity it allows. Here we will load a dataset of all the major cities in the world with their population counts over time:

In [ ]:
cities = pd.read_csv('./assets/cities.csv', encoding="ISO-8859-1")
population = gv.Dataset(cities, kdims=['City', 'Country', 'Year'])
cities.head()

Now we can convert this dataset to a set of points mapped by the latitude and longitude and containing the population, country and city as values. The longitudes and latitudes in the dataframe are supplied in simple Plate Carree coordinates, which we will need to declare (as the values are not stored with any associated units). The ``.to`` conversion interface lets us do this succinctly. Note that since we did not assign the Year dimension to the points key or value dimensions, it is automatically assigned to a HoloMap, rendering the data as an animation using a slider widget:

In [ ]:
%%opts Overlay [width=600 height=350] 
%%opts Points (size=0.005 cmap='viridis') [tools=['hover'] size_index=2 color_index=2]
(gv.WMTS(tiles['Wikipedia']) *\
population.to(gv.Points, kdims=['Longitude', 'Latitude'],
              vdims=['Population', 'City', 'Country'], crs=ccrs.PlateCarree()))

And because this is a fully interactive Bokeh plot, you can now hover over each datapoint to see all of the values associated with it (name, location, etc.), and you can zoom and pan using the tools provided.  Each time, the map tiles should seamlessly update to provide additional detail appropriate for that zoom level.


## Choropleths

The tutorial on [Geometries](Geometries.ipynb) covers working with shapefiles in more detail but here we will quickly combine a shapefile with a pandas DataFrame to plot the results of the EU Referendum in the UK. We begin by loading the shapefile and then us ``pd.merge`` by combining it with some CSV data containing the referendum results:

In [ ]:
import geopandas as gpd
geometries = gpd.read_file('./assets/boundaries/boundaries.shp')
referendum = pd.read_csv('./assets/referendum.csv')
gdf = gpd.GeoDataFrame(pd.merge(geometries, referendum))

Now we can easily pass the GeoDataFrame to a Polygons object and declare the ``leaveVoteshare`` as the first value dimension which it will color by:

In [ ]:
%%opts Polygons [tools=['hover'] width=450 height=600 color_index='leaveVoteshare' colorbar=True toolbar='above' xaxis=None yaxis=None]
gv.Polygons(gdf, vdims=['name', 'leaveVoteshare'])

### Images

The Bokeh backend also provides basic support for working with images. In this example we will load a very simple Iris Cube and display it overlaid with the coastlines feature from Cartopy. Note that the Bokeh backend does not project the image directly into the web Mercator projection, instead relying on regridding, i.e. resampling the data using a new grid. This means the actual display may be subtly different from the more powerful image support for the matplotlib backend, which will project each of the pixels into the chosen display coordinate system without regridding.

In [ ]:
%%opts Overlay [width=600 height=500] Image (cmap='viridis') Feature (line_color='black')
dataset = xr.open_dataset('./sample-data/pre-industrial.nc')
air_temperature = gv.Dataset(dataset, kdims=['longitude', 'latitude'],
                             group='Pre-industrial air temperature', vdims=['air_temperature'],
                             crs=ccrs.PlateCarree())
air_temperature.to.image() * gf.coastline()

Support for other projections is eventually planned for Bokeh, but meanwhile you should usually be able to use either backend interchangeably as long as you use web Mercator coordinates for display, and the additional interactivity provided by Bokeh is often very useful!